# Imports

In [12]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.metrics import Precision, Recall
from keras.applications.vgg19 import VGG19
from keras.callbacks import ModelCheckpoint
from keras.layers import Flatten, Dropout, Dense
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator


physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

IndexError: list index out of range

# Prepare Data

In [2]:
images = []
labels = []

data = pd.read_csv('data/instagram_data.csv')
for image in os.listdir("data/images"):
    if "_".join(image.split('_')[1:]).split('.')[0] in data.imagename.tolist():
        row = data.loc[data['imagename'] == "_".join(image.split('_')[1:]).split('.')[0]]
        images.append(image)
        if row['Q5A.  If yes to Q5, what type of Asian'].values[0] == 1:
            labels.append(1)
        else:
            labels.append(0)

pd.DataFrame({"filename": images, "label": labels}).to_csv('data/processed_instagram_data.csv', index=False)

# Import data

In [9]:
IMG_DIR = 'data/images'
IMG_SIZE = 224
CHANNELS = 3
NUM_CLASSES = 2
BATCH_SIZE = 32
NUM_EPOCHS = 10

train_data = pd.read_csv('data/processed_instagram_data.csv', dtype='string')
X_length = len(train_data)
Y = train_data[['label']]

skf = StratifiedKFold(n_splits = 5, shuffle = True) 

datagen = ImageDataGenerator(rescale=1./255, horizontal_flip = True,
                                   preprocessing_function=tf.keras.applications.vgg19.preprocess_input)

# Model Definition

In [10]:
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

def new_model():
    vgg = VGG19(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, CHANNELS))
    vgg.trainable = False

    model = Sequential()
    model.add(vgg)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    return model

# Training

In [14]:
VALIDATION_ACCURACY = []
VALIDATION_PRECISION = []
VALIDATION_RECALL = []
VALIDATION_LOSS = []

save_dir = 'saved_models/'
fold_var = 1

for train_index, val_index in skf.split(np.zeros(X_length),Y):
	training_data = train_data.iloc[train_index]
	validation_data = train_data.iloc[val_index]
	
	train_data_generator = datagen.flow_from_dataframe(training_data, directory = IMG_DIR,
						    x_col = "filename", y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
						    shuffle = True)
	valid_data_generator  = datagen.flow_from_dataframe(validation_data, directory = IMG_DIR,
							x_col = "filename", y_col = "label",
                            target_size = (IMG_SIZE, IMG_SIZE),
						    shuffle = True)
	
	model = new_model()
	model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=[Precision(), Recall(), 'accuracy'])

	checkpoint = ModelCheckpoint(save_dir+get_model_name(fold_var), 
							monitor='val_accuracy', verbose=1, 
							save_best_only=True, mode='max')
	
	callbacks_list = [checkpoint]

	history = model.fit(train_data_generator,
			    epochs=NUM_EPOCHS,
			    callbacks=callbacks_list,
			    validation_data=valid_data_generator)

	model.load_weights("saved_models/model_"+str(fold_var)+".h5")
	
	results = model.evaluate(valid_data_generator)
	results = dict(zip(model.metrics_names,results))
	
	VALIDATION_ACCURACY.append(results['accuracy'])
	VALIDATION_PRECISION.append(results['precision'])
	VALIDATION_RECALL.append(results['recall'])
	VALIDATION_LOSS.append(results['loss'])
	
	fold_var += 1

Found 2983 validated image filenames belonging to 2 classes.
Found 746 validated image filenames belonging to 2 classes.
Epoch 1/10
65/94 [===================>..........] - ETA: 49s - loss: 0.8727 - precision: 0.8341 - recall: 0.8341 - accuracy: 0.8341

KeyboardInterrupt: 